<a href="https://colab.research.google.com/github/srini229/EE5333_tutorials/blob/master/Partition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 43.9 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.16.0
    Uninstalling cffi-1.16.0:
      Successfully uninstalled cffi-1.16.0


## Partitioning

* Kernighan Lin Algorithm for bi-partitioning ($V'$) :
  + $G=(V,E)$
  + $A$, $B$ $\subset V$
  + $A \cup B = V$
  + $A\cap B = ∅$
  + $|A| = |B| = \dfrac{|V|}{2}$
  + Flowchart:

    <img src="https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/part/fig/KL_flowchart.jpg" width=698 height=612 />


In [ ]:
# Vertex class to hold the partition index, neighbours, EA, EB and D values
class Vertex:
  def __init__(self, i, part):
    self._id = i
    self._nbrs = []
    self._part = part
    self._ea = 0
    self._eb = 0
    self._d  = 0
  def reset(self, part):
    (self._part, self._ea, self._eb, self._d) = (part, 0, 0, 0)
  def __str__(self):
    return '(' + str(self._id) + ',' + str(self._part) + ',' + str(self._ea) + ',' + str(self._eb) + ',' + str(self._d) + ',' + str([i._id for i in self._nbrs]) + ')'
  def __repr__(self):
    return str(self)

# clear the partition, EA, EB and D values
# do this at the beginning of every iteration
def reset(V, A, B):
  for j in range(2):
    partition = A if (0 == j) else B
    for i in partition:
      V[i].reset(j)
  for v in V:
    assert(v._part == 0 or v._part == 1)
    for n in v._nbrs:
      if n._part == 0:
        v._ea += 1
      else:
        v._eb += 1
  for v in V:
    v._d = (v._ea - v._eb) if (v._part == 1) else (v._eb - v._ea)

# Choose the pair whose swap has the maximum gain in number of cuts
def findMaxGain(V, Ap, Bp, E):
  (amax, bmax, gmax) = (-1, -1, -2 * len(E) - 1)
  for a in Ap:
    for b in Bp:
      g = V[a]._d + V[b]._d - (2 if (min(a,b), max(a,b)) in E else 0)
      if gmax < g:
        (amax, bmax, gmax) = (a, b, g)
  assert(amax >= 0 and bmax >= 0)
  return (amax, bmax, gmax)

# update the E and D for only the affected neighbours of a and b
def updateED(V, a, b):
  V[a]._part = 1
  V[b]._part = 0
  for i in [a,b]:
    for n in V[i]._nbrs:
      if i == a:
        n._ea -= 1
        n._eb += 1
      else:
        n._ea += 1
        n._eb -= 1
      n._d = (n._ea - n._eb) if (n._part == 1) else (n._eb - n._ea)

# N is the number of vertices; vertices are {0, 1,... N-1}
# E is the list of edges
# E : list of edges; edge = unordered pair of vertices
# Return value : two sets A, B and the count of number of cuts
def KLPart(N, E):
  if N%2: N+= 1 # make N even if its odd by adding a single no-neighbour vertex
  V = [Vertex(i, -1) for i in range(N)]
  for e in E:
    if e[0] > e[1]: e = (e[1], e[0])
    else: e = (e[0], e[1])
  E = set(E)
  for e in E:
    V[e[0]]._nbrs.append(V[e[1]])
    V[e[1]]._nbrs.append(V[e[0]])
  import random
  Vc = V[:]
  partLen = N//2
  random.shuffle(Vc) # randomly initialize A and B

  A = {Vc[i]._id for i in range(partLen)}
  B = {Vc[i]._id for i in range(partLen, N)}

  print(A)
  print(B)
  maxGain = 1
  while maxGain >= 0:
    Ap, Bp= A.copy(), B.copy()
    reset(V, A, B)
    G, S = [], []
    for p in range(partLen):
      (a, b, g) = findMaxGain(V, Ap, Bp, E)
      updateED(V, a, b)
      Ap.remove(a)
      Bp.remove(b)
      G.append(g)
      S.append((a, b))
    for i in range(1, len(G)):
      G[i] += G[i-1]
    maxGain = max(G)
    maxIndex = G.index(maxGain)
    if maxGain > 0:
      for (a, b) in S[0:maxIndex + 1]:
        A.remove(a)
        B.remove(b)
        A.add(b)
        B.add(a)
    else:
      break

  cut = 0
  for a in A:
    for b in B:
      if (min(a, b), max(a,b)) in E:
        cut += 1
  return (A, B, cut)

In [ ]:
print(KLPart(8, [(0,1), (0,4), (0,5), (1,4), (1,5), (4,5), (2,3), (2,6), (2,7), (3,6), (3,7), (6,7), (2,5)]))

{0, 3, 5, 7}
{1, 2, 4, 6}
({2, 3, 6, 7}, {0, 1, 4, 5}, 1)


## Bipartitioning using ILP
+ $x_v$ is the indicator variable for $v$ being in $A$
+ $x_{u,v}$ is the indicator variable for $(u,v)\in E$ being cut
+ <ul>
$\begin{align}
        x_{u,v} = x_u \oplus x_v
\end{align}$
</ul>

+ Objective: $\min\limits_{x_v, x_{u,v}} \sum\limits_{(u,v)\in E}x_{u,v}$
+ Subject to constraints:
<ul>
$\begin{align}
\sum_{v\in V} x_v&=\frac{|V|}{2}\\
x_u - x_v &\leq x_{u,v}, &\forall (u,v) \in E\\
x_v - x_u &\leq x_{u,v}, &\forall (u,v) \in E\\
x_u + x_v &\geq x_{u,v}, &\forall (u,v) \in E\\
x_u + x_v + x_{u,v} &\leq 2, &\forall (u,v) \in E\\
x_v &\in \{0, 1\}, &\forall v \in V\\
x_{u,v} &\in \{0, 1\}, &\forall (u,v) \in E
\end{align}$
</ul>


In [ ]:
def bipartition(N, E):
  import mip
  model = mip.Model("Bi-partition")
  x = [model.add_var(f"x_{u}", var_type = mip.BINARY) for u in range(N)]
  x_uv = [model.add_var(f"x_{u}_{v}", var_type = mip.BINARY) for u,v in E]
  model.verbose = 0
  model.objective = mip.minimize(mip.xsum(x_uv))
  model += (mip.xsum(x) == N//2)
  for e, (u,v) in enumerate(E):
  # xor constraints
    model += (x[u] - x[v] <= x_uv[e])
    model += (x[v] - x[u] <= x_uv[e])
    model += (x[u] + x[v] >= x_uv[e])
    model += (x[u] + x[v] + x_uv[e] <= 2)

  model.write("bipartition.lp")
  model.optimize()
  if model.status == mip.OptimizationStatus.OPTIMAL:
    A = [i for i in range(N) if x[i].x >= 0.9]
    B = [i for i in range(N) if x[i].x < 0.9]
    return (A, B, model.objective.x)
  return None



In [ ]:
print(bipartition(8, [(0,1), (0,4), (0,5), (1,4), (1,5), (4,5), (2,3), (2,6), (2,7), (3,6), (3,7), (6,7), (2,5)]))

([2, 3, 6, 7], [0, 1, 4, 5], 1.0)


In [ ]:
def random_graph(N, fns):
  import networkx as nx
  import time
  g = nx.erdos_renyi_graph(N, 0.4)
  G = []
  for fn in fns:
    t = time.time()
    (A,B,c) = fn(N, g.edges)
    print("runtime : ", time.time() - t, 'cut size : ', c)
    import graphviz
    gv = graphviz.Graph()
    A = list(A)
    B = list(B)
    for i in range(len(A)):
      gv.node(str(A[i]), color='red', shape='circle')
    for i in range(len(B)):
      gv.node(str(B[i]), color='blue', shape='circle')
    for e in g.edges:
      gv.edge(str(e[0]), str(e[1]), splines='line')
    #gv.engine = 'patchwork'
    G.append(gv)
  return G

In [ ]:
G = random_graph(10, [KLPart, bipartition])
#from IPython.display import display
#display(G[0], G[1])

{0, 4, 5, 7, 9}
{1, 2, 3, 6, 8}
runtime :  0.0001513957977294922 cut size :  3
runtime :  0.041370391845703125 cut size :  3.0


In [ ]:
!cat bipartition.lp

\Problem name: Bi-partition

Minimize
OBJROW: x_0_4 + x_0_5 + x_0_9 + x_1_2 + x_1_5 + x_1_6 + x_2_4 + x_2_6 + x_3_8 + x_4_6
 + x_4_9 + x_5_6 + x_5_7 + x_6_9 + x_7_9 + x_8_9
Subject To
constr(0):  x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9
 = 5
constr(1):  x_0 - x_4 - x_0_4 <= -0
constr(2):  - x_0 + x_4 - x_0_4 <= -0
constr(3):  x_0 + x_4 - x_0_4 >= -0
constr(4):  x_0 + x_4 + x_0_4 <= 2
constr(5):  x_0 - x_5 - x_0_5 <= -0
constr(6):  - x_0 + x_5 - x_0_5 <= -0
constr(7):  x_0 + x_5 - x_0_5 >= -0
constr(8):  x_0 + x_5 + x_0_5 <= 2
constr(9):  x_0 - x_9 - x_0_9 <= -0
constr(10):  - x_0 + x_9 - x_0_9 <= -0
constr(11):  x_0 + x_9 - x_0_9 >= -0
constr(12):  x_0 + x_9 + x_0_9 <= 2
constr(13):  x_1 - x_2 - x_1_2 <= -0
constr(14):  - x_1 + x_2 - x_1_2 <= -0
constr(15):  x_1 + x_2 - x_1_2 >= -0
constr(16):  x_1 + x_2 + x_1_2 <= 2
constr(17):  x_1 - x_5 - x_1_5 <= -0
constr(18):  - x_1 + x_5 - x_1_5 <= -0
constr(19):  x_1 + x_5 - x_1_5 >= -0
constr(20):  x_1 + x_5 + x_1_5 <= 2
const

## Bipartitioning using ILP - II
+ $x_{v,A}$ is the indicator variable for $v$ being in $A$
+ Similarly define $x_{v,B}$
+ $x_{u,v,A}$ is the indicator variable for $(u,v)\in E$ being contained in $A$

+ Objective: $\max\limits_{x_{u,v,i}} \sum\limits_{(u,v)\in E}x_{u,v,A} + x_{u,v,B}$
+ Subject to constraints:
<ul>
$\begin{align}
\sum_{v\in V} x_{v,i}&=\frac{|V|}{2}, &\forall v \in V\\
x_{v,A}, x_{v,B} &\in \{0, 1\}, &\forall v \in V\\
x_{u,v,A} &\leq x_{u,A}, &\forall (u,v) \in E\\
x_{u,v,A} &\leq x_{v,A}, &\forall (u,v) \in E\\
x_{v,A} + x_{v,B} &=1 , &\forall v \in V\\
x_{u,v,A}, x_{u,v,B} &\in \{0, 1\}, &\forall (u,v) \in E
\end{align}$
</ul>

In [ ]:
def bipartition2(N, E):
  k = 2
  import mip
  model = mip.Model(f"{k}-way partition")
  x = {u:[model.add_var(f"x_{u}_{i}", var_type = mip.BINARY) for i in range(k)] for u in range(N)}
  x_e = [[model.add_var(f"x_{e[0]}_{e[0]}_{i}", var_type = mip.BINARY) for i in range(k)] for e in enumerate(E)]
  model.verbose = 0
  model.objective = mip.maximize(mip.xsum(x_e[j][i] for j,e in enumerate(E) for i in range(k)))

  for u in range(N):
    model += mip.xsum(x[u]) == 1

  for i in range(k):
    model += mip.xsum(x[u][i] for u in range(N)) == N//k

  for j,e in enumerate(E):
    for i in range(k):
      model += x_e[j][i] <= x[e[0]][i]
      model += x_e[j][i] <= x[e[1]][i]

  model.write(f"partition{k}.lp")
  model.optimize()
  sol = [list() for i in range(k)]

  if model.status == mip.OptimizationStatus.OPTIMAL:
    for u in range(N):
      for i in range(k):
        if round(x[u][i].x) == 1: sol[i].append(u)

    return (sol[0], sol[1], len(E)- model.objective.x)
  return None

In [ ]:
G = random_graph(20, [KLPart, bipartition, bipartition2])
#from IPython.display import display
#display(G[0], G[1], G[2])

{1, 2, 6, 8, 9, 10, 13, 16, 17, 18}
{0, 3, 4, 5, 7, 11, 12, 14, 15, 19}
runtime :  0.000453948974609375 cut size :  24
runtime :  1.4052271842956543 cut size :  24.0
runtime :  7.571538209915161 cut size :  24.0


## LEF DEF Parsing

- Install the LEFDEFParser from the the wheel file : [LEFDEFParser-0.1-cp310-cp310-linux_x86_64.whl](https://github.com/srini229/EE5333_tutorials/blob/master/parser/LEFDEFParser-0.1-cp310-cp310-linux_x86_64.whl)
- Download example LEF and DEF files: [Nangate.lef](https://github.com/srini229/EE5333_tutorials/blob/master/parser/Nangate.lef) and [example.def](https://github.com/srini229/EE5333_tutorials/blob/master/parser/example.def)

    <img src="https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/part/fig/example_cir.png" width=340 height=195 />



In [ ]:
!pip install --break-system-packages https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/LEFDEFParser-0.1-cp311-cp311-linux_x86_64.whl
!rm *.{lef,def}
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/{Nangate.lef,example.def}
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/sample.{lef,def}

Defaulting to user installation because normal site-packages is not writeable
  Using cached https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/LEFDEFParser-0.1-cp311-cp311-linux_x86_64.whl (554 kB)


In [ ]:
class Vertex:
  def __init__(self, name, area, nbrs):
    self._name = name
    self._area = area
  def __repr__(self):
    return self._name + f" ({round(self._area,2)})"

def loadNetlist(leffile = None, deffile = None):
  import LEFDEFParser as LDP
  l = LDP.LEFReader()
  areaLookup = dict()
  if leffile:
    l.readLEF(leffile)
    areaLookup = {m.name():(m.xdim()*m.ydim()*1.e-6) for m in l.macros()}
  vertices = dict()
  edges = dict()
  if deffile:
    d = LDP.DEFReader()
    d.readDEF(deffile)
    vertices = {c.name() : Vertex(c.name(), areaLookup.get(c.macro(), None), list()) for c in d.components()}
    edges = {n.name():[vertices[p[0]] for p in n.pins() if p[0] != 'PIN'] for n in d.nets()}
  delE = list()
  for e in edges:
    if len(edges[e]) <= 1:
      delE.append(e)
  for e in delE: del edges[e]
  return vertices, edges

## $k$-way hypergraph partitioning using ILP
+ Hypergraph $H(V,E)$
+ $x_{v,i}$ is the indicator variable for $v$ being in partition $V_i$
+ $x_{e,i}$ is the indicator variable for $e\in E$ being contained in $V_i$

+ Objective: $\max\limits_{x_{e,i}} \sum\limits_{e\in E}\sum\limits_{i=1}^k x_{e,i}$
+ Subject to constraints:
<ul>
$\begin{align}
x_{v,i} &\in \{0, 1\}, &\forall v \in V, \forall i \in \{1,2,\ldots, k\}\\
x_{e,i} &\in \{0, 1\}, &\forall e \in E, \forall i \in \{1,2,\ldots, k\}\\
\sum\limits_{i=1}^k x_{v,i} &=1 , &\forall v \in V\\
\sum_{v\in V} area(v)\cdot x_{v,i}&\leq Area_{max} &\forall v \in V\\
\sum_{v\in V} area(v)\cdot x_{v,i}&\geq Area_{min} &\forall v \in V\\
x_{e,i} &\leq x_{v,i}, &\forall e \in E, \forall v~\text{connected by}~e\\
\end{align}$
</ul>

In [ ]:
def partition(V, E, k, Amin, Amax):
  import mip
  model = mip.Model(f"{k}-way partition")
  x = {u:[model.add_var(f"x_{u}_{i}", var_type = mip.BINARY) for i in range(k)] for u in V}
  x_e = {e:[model.add_var(f"x_{e}_{i}", var_type = mip.BINARY) for i in range(k)] for e in E}
  model.verbose = 0
  model.objective = mip.maximize(mip.xsum(x_e[e][i] for e in E for i in range(k)))

  for u in V:
    model += mip.xsum(x[u]) == 1

  for i in range(k):
    model += mip.xsum(V[u]._area*x[u][i] for u in V) >= Amin
    model += mip.xsum(V[u]._area*x[u][i] for u in V) <= Amax

  for e in E:
    for i in range(k):
      for v in E[e]:
        model += x_e[e][i] <= x[v._name][i]

  model.write(f"partition{k}.lp")
  model.optimize()
  sol = [list() for i in range(k)]

  if model.status == mip.OptimizationStatus.OPTIMAL:
    for u in V:
      for i in range(k):
        if round(x[u][i].x) == 1: sol[i].append(V[u])

    return (sol, len(E)- model.objective.x)
  return None


In [ ]:
import time
k=2
V,E = loadNetlist('Nangate.lef', 'example.def')
Atotal = sum(V[u]._area for u in V)
maxCellArea = max(V[u]._area for u in V)
print(Atotal, round(maxCellArea,2))
t = time.time()
sol, numcuts = partition(V, E, k, Atotal/k - maxCellArea/6, Atotal/k + maxCellArea/6)
print("runtime :", time.time() - t)
print("number of cuts :", round(numcuts))
for part in sol:
  print(part, round(sum([x._area for x in part]),2))

18.088 3.19
runtime : 0.006925344467163086
number of cuts : 2
[c (3.19), d (3.19), f (2.13)] 8.51
[a (3.19), b (3.19), e (3.19)] 9.58


In [ ]:
import time
k=2
V,E = loadNetlist('sample.lef', 'sample.def')
Atotal = sum(V[u]._area for u in V)
maxCellArea = max(V[u]._area for u in V)
print(Atotal, round(maxCellArea,2))
t = time.time()
sol, numcuts = partition(V, E, k, Atotal/k - maxCellArea/6, Atotal/k + maxCellArea/6)
print("runtime :", time.time() - t)
print("number of cuts :", round(numcuts))
for part in sol:
  print(part, round(sum([x._area for x in part]),2))

250.344 25.99
runtime : 0.005991935729980469
number of cuts : 0
[inst2015 (10.94), inst2591 (15.05), inst2908 (9.58), inst3502 (13.68), inst4189 (15.05), inst4597 (8.21), inst5333 (12.31), inst6286 (10.94), inst6458 (17.78), inst6050 (8.21)] 121.75
[inst3428 (8.21), inst3444 (8.21), inst4132 (8.21), inst4183 (25.99), inst4062 (12.31), inst4678 (5.47), inst4382 (8.21), inst5638 (12.31), inst5275 (9.58), inst5821 (6.84), inst5195 (12.31), inst7234 (10.94)] 128.59
